In [22]:
# -*- coding: utf-8 -*-
"""
Outlook folder reader:
 Read specified folder and save items in the folder to csv

original:
https://www.laurivan.com/python-and-outlook-an-example/
see:
https://docs.python.jp/3/using/windows.html#pywin32
http://timgolden.me.uk/pywin32-docs/contents.html
http://timgolden.me.uk/pywin32-docs/html/com/win32com/HTML/QuickStartClientCom.html
http://fum125.hatenablog.com/entry/2016/11/12/233457
https://msdn.microsoft.com/ja-jp/VBA/Outlook-VBA/articles/recipient-type-property-outlook
https://docs.python.jp/3/library/csv.html?highlight=csv
https://msdn.microsoft.com/ja-jp/vba/outlook-vba/articles/propertyaccessor-object-outlook
"""
import win32com.client

DEBUG = False

# Outlook オブジェクト作成

In [23]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# サポートメソッド

In [24]:
def items(outlook_items_object):
    array_size = outlook_items_object.Count
    for item_index in range(1,array_size+1):
        yield (item_index, outlook_items_object[item_index])

def prop(outlook_object):
    return sorted( outlook_object._prop_map_get_.keys() )


# フォルダーのパース

In [25]:
roots = outlook.Folders

In [26]:
def parseFolders(folders, depth=0, recursive=True):
    if not folders:
        return
    for index, folder in items(folders):
        print( " "*depth*2, u"(%i) [%s] [%s]" % (index, folder.Name, folder))
        if recursive:
            parseFolders(folder.Folders, depth + 1, recursive)


In [27]:
#parseFolders(roots, recursive=True)

# 特定のフォルダのアイテム表示

In [28]:
def search(folders, queryPath):
    if DEBUG:
        print(queryPath)
    if not folders:
        return None
    if len(queryPath)<=0:
        return None
    for index, folder in items(folders):
        if folder.Name == queryPath[0]:
            if len(queryPath)==1:
                return folder
            else:
                return search(folder.Folders, queryPath[1:])
    return None

def printItems(mailItems):
    for index, mailItem in items(mailItems):
        mailFrom = mailItem.SenderEmailAddress + ";"
        mailTo = ""
        mailCc = ""
        for i, recipient in items(mailItem.Recipients):
            if recipient.Type == win32com.client.constants.olTo:
                mailTo += recipient.Address + ";"
            if (recipient.Type == win32com.client.constants.olCC or recipient.Type == win32com.client.constants.olBCC):
                mailCc += recipient.Address + ";"
            if recipient.Type == win32com.client.constants.olOriginator:
                mailFrom += recipient.Address + ";"
        mailSubj = mailItem.Subject
        mailSentDate = mailItem.SentOn
        print("[%s]\t[%s]\t[%s]\t[%s]\t[%s]" % (mailFrom, mailTo, mailCc, mailSubj, mailSentDate))
    return

In [29]:
folder = search(roots, 'Outlook データ ファイル\\受信トレイ'.split('\\'))
print(folder.Name)
printItems(folder.Items)


受信トレイ


# 特定のフォルダのアイテムをCSV保存

In [30]:
import csv

def getMailItemHeader(mailItem):
    PropName = "http://schemas.microsoft.com/mapi/proptag/0x007D001E"
    return mailItem.PropertyAccessor.GetProperty(PropName)

def saveItems(mailItems, saveFileName):
    with open(saveFileName, 'w', newline='', encoding='utf-8', errors='replace') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["from", "to", "cc", "sentDate", "subject", "body", "internet mail header"])

        for index, mailItem in items(mailItems):
            mailFrom = mailItem.SenderEmailAddress + ";"
            mailTo = ""
            mailCc = ""
            for i, recipient in items(mailItem.Recipients):
                if recipient.Type == win32com.client.constants.olTo:
                    mailTo += recipient.Address + ";"
                if (recipient.Type == win32com.client.constants.olCC or recipient.Type == win32com.client.constants.olBCC):
                    mailCc += recipient.Address + ";"
                if recipient.Type == win32com.client.constants.olOriginator:
                    mailFrom += recipient.Address + ";"
            mailSubj = mailItem.Subject
            mailSentDate = mailItem.SentOn
            mailBody = mailItem.Body
            mailHeader = getMailItemHeader(mailItem)
            writer.writerow([mailFrom, mailTo, mailCc, mailSentDate, mailSubj, mailBody, mailHeader])
    return

In [31]:
folder = search(roots, 'Outlook データ ファイル\\受信トレイ'.split('\\'))
print(folder.Name)
saveItems(folder.Items, 'test.csv')

受信トレイ
